# Histo1D

The **Histo1D** class provides the same features as **ROOT**'s **TH1D** class but with extended functionality and improved ease of use. So make sure to also consult the [ROOT documentation](https://root.cern.ch/doc/master/classTH1D.html).

Again we start by creating a test sample that we can play with:

In [ ]:
import ROOT
from mephisto import IOManager, Histo1D

IOManager.CreateTestSample("testsample.root", tree="TestTree", nbranches=10, nevents=1e4)

Now let's create a Histo1D, fill it with some dummy data and draw it to a canvas. You can use equidistant bins or define your own intervalls, you decide!

In [ ]:
## equidistant bins:
h = Histo1D("h", "MyHisto", 50, 0.0, 10.0)

## variable bins:
# binlowedges = [0, 1, 2, 2.5, 3, 3.5, 4, 4.5, 5, 6, 7, 8, 10]
# h = Histo1D("h", "MyHisto", binlowedges)

h.Fill("testsample.root", tree="TestTree", varexp="branch_5", cuts=["branch_6>2", "branch_7<8"], weight="branch_10")

c = ROOT.TCanvas()
c.Draw()
h.Draw("HIST")


The ```Fill``` method uses the ```FillHistogram``` method of the **IOManager**. If you're going to fill multiple histograms from the same TTree, consider using the **Factory** subclass of the IOManager instead. If the last sentence didn't make any sense to you, have a look at IOManager tutorial!

The above plot looks quite ugly. There is no axis title, legend or errorband. Instead of drawing it to a canvas, let's use Histo1D's ```Print``` method to create a PNG file of the plot.

In [ ]:
try:
    del c  # delete it so that Jupyter doesn't get confused...
except NameError:
    pass

h.Print("myhisto.png")

# view in in Jupyter:
from IPython.display import Image
Image(filename='myhisto.png')

That looks at lot better already! Note that the y-axis is plotted logarithmically by default.

So far so good, but we could still tweak it a bit:

In [ ]:
h.Print("myhisto.png",
        logy=False,
        linewidth=3,
        drawerrorband=True,
        errorbanddrawoption="E2",
        errorbandfillstyle=3151,
        xtitle="m_{T}(#mu, E_{T}^{miss})",
        xunits="TeV")

Image(filename='myhisto.png')

We have now set the y-axis to a linear scale, increased the line thickness of the histogram, added an errorband (computed by ```TH1::Sumw2```) and added custom x-axis title and associated units. It would of course be possible to customize the plot even further: You can give ```Print``` or the Histo1D constructor itself any **property** associated to a Histo1D.

### What are properties?

The short answer is, (almost) any attribute of a class for which there is a setter method. For example, a Histo1D object has a setter called ```SetLineWidth``` that takes one integer as an argument. Thus ```linewidth``` is a *property* of a Histo1D object and you can define it either by using the setter method or give it as a keyword argument to the class constructor, the ```Print``` method or the ```DeclareProperty```/```DeclareProperties``` method. If you'd like to change a properties of the errorband associated to the histogram you simply need to add the prefix ```errorband``` to the name of the property, like we did by ```errorbandfillstyle=3151``` above. The same hold for properties of the associated Legend (inherits from TLegend): Just add the prefix ```legend``` in fron of the property name!

If a setter required more than one argument, like ```SetFillColorAlpha``` for example, the value associated to the given keyword argument must be a ```list``` or ```tuple```. Color values can be either integers (i.e. also of the form ```ROOT.kColor```) or strings containing a hex-code.

There are some setters, like ```SetBinContent``` for example, that are not associated with a property for safety reasons. The ```name``` property also does not exist in order to keep track of the histogram that are created by the user. Since the methods like ```SetXTitle``` exist for both in the Histo1D and in the Pad (inherits from TPad) class, the associated properties have been made exclusive for Pad objects. You can steer it however via the ```Print``` method which propagates it to the Pad in which the histogram is drawn.

With this fresh knowlegde about Histo1D properties, let's create a dictionary with all the properties we'd like to apply to our histogram:

In [ ]:
props = {
    "drawoption":           "HIST",
    "linestyle":            1,
    "linewidth":            1,
    "linecolor":            "#000000",
    "linealpha":            1.0,
    "fillstyle":            1001,
    "fillcolor":            "#2b2bff",
    "fillalpha":            1.0,
    "drawerrorband":        True,
    "errorbanddrawoption":  "E2",
    "errorbandfillstyle":   3151,
    "errorbandfillalpha":   1.0,
    "legenddrawoption":     "F"
}

h.Print("myhisto.png", **props)

Image(filename='myhisto.png')

As this style is quite common for histograms representing a *background* process, there is already a ```template``` for it. If you whish to use it and, for example, just want to change the fill color to green, it's as easy as that:

In [ ]:
h.Print("myhisto.png", template="background", fillcolor=ROOT.kGreen)

Image(filename='myhisto.png')

You can find other template for Histo1D object and other Mephisto classes in the ```mephisto/templates``` folder. If have a set of properties you use regularly, consider adding it to the respective JSON file!

Finally, if you want to save the histogram into a directory which does not exist yet, just add the ```mkdir=True``` flag and Mephisto will take care of it.

In [ ]:
h.Print("plots/myhisto.png", mkdir=True)

That's it! Now you know the basics of Histo1Ds.